## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
import os
import math

## Graph Visualization

In [2]:
df_dist = pd.read_csv("distances.csv", header=None)
df_dist.columns=["Departure_City", "Destination_City", "Distance"]
df_dist["Departure_City"] = df_dist.Departure_City.apply(lambda x: "Adapazari" if x == "Sakarya" else x)
df_dist["Destination_City"] = df_dist.Destination_City.apply(lambda x: "Adapazari" if x == "Sakarya" else x)
df_dist.head()

,Departure_City,Destination_City,Distance
0,Istanbul,Izmit,103
1,Izmit,Adapazari,53
2,Adapazari,Bolu,125
3,Bolu,Ankara,187
4,Ankara,Kirikkale,73


In [3]:
Cities_Graph = nx.from_pandas_edgelist(df_dist,
                           source="Departure_City",
                           target="Destination_City",
                           edge_attr=True)

In [4]:
net = Network(notebook=True)
net.from_nx(Cities_Graph)
net.show("graph.html")

## Route Finding Algorithm

In [5]:
def calculateRoute(start_end_city:list, distances_file:str="distances.csv"):
    #Check if the file exists
    assert os.path.isfile(distances_file), "Distances file does not exists. Your input is: " + str(distances_file)
    
    # Check if the input list has starting and destination cities.
    assert len(start_end_city) == 2, "Input city length must be equal to 2. Your input is: "+ str(start_end_city)
    
    # Assign cities
    starting_city = start_end_city[0]
    destination_city = start_end_city[1]
    
    # Check types of inputs
    assert type(starting_city) == str, "Starting city must be a string.You entered: " + str(type(starting_city))
    assert type(destination_city) == str, "Destination city must be a string. You entered: " + str(type(destination_city))
    
    # Check if input cities are same
    assert starting_city != destination_city, "Starting and destination city cannot be equal."
    
    # Load distance.csv
    df_dist = pd.read_csv(distances_file, header=None)
    df_dist.columns=["Departure_City", "Destination_City", "Distance"]
    
    # Generate total map to move backard and forward between nodes
    df_dist_reverse = df_dist[["Destination_City", "Departure_City", "Distance"]]
    df_dist_reverse.columns = ["Departure_City", "Destination_City", "Distance"]
    df_total_dist = pd.concat([df_dist_reverse, df_dist], axis=0)
    
    # Convert Sakarya to Adapazari since they represent same city
    df_total_dist["Departure_City"] = df_total_dist.Departure_City.apply(lambda x: "Adapazari" if x == "Sakarya" else x)
    df_total_dist["Destination_City"] = df_total_dist.Destination_City.apply(lambda x: "Adapazari" if x == "Sakarya" else x)
    
    # unload unnecessary dataframes
    del df_dist_reverse
    del df_dist
    
    # Creating city list
    city_list = list(dict.fromkeys(list(set(df_total_dist["Departure_City"]))))
    
    # Check if input cities are given properly.
    assert starting_city in city_list, "Starting city is not in cities. It must be one of the " + str(city_list)
    assert destination_city in city_list, "Starting city is not in cities. It must be one of the " + str(city_list)
    
    
    # Generating dictionary to keep all coonections of each node with their distances.
    City_Tree = dict.fromkeys(list(set(df_total_dist["Departure_City"])))
    for ea_city in City_Tree:
        connected_cities = df_total_dist[df_total_dist.Departure_City == ea_city]["Destination_City"].to_numpy()
        city_distances = df_total_dist[df_total_dist.Departure_City == ea_city]["Distance"].to_numpy()

        City_Tree[ea_city] = {}
        for i,ea_connection in enumerate(connected_cities):
            City_Tree[ea_city][ea_connection] = city_distances[i]
    # Route        
    route = []       
    shortest_route = {}
    former_nodes = {}
    
    
    # Assign infinity to unvisited cities
    for node in City_Tree:
        shortest_route[node] = math.inf
    shortest_route[starting_city] = 0
    
    # Find shortest path by iterating connected nodes
    while City_Tree:
        shortest_node = None
        
        for node in City_Tree:
            if shortest_node is None:
                shortest_node = node
            elif shortest_route[node] < shortest_route[shortest_node]:
                shortest_node = node
                
        # Iterate every node and their chields   
        for child_node, distance in City_Tree[shortest_node].items():
            if distance + shortest_route[shortest_node] < shortest_route[child_node]:
                shortest_route[child_node] = distance + shortest_route[shortest_node]
                former_nodes[child_node] = shortest_node
        
        # Since the node is visited it is removed.
        City_Tree.pop(shortest_node)
    
    # Assign destination_city to current_node to move from the end to beginning
    current_node = destination_city
    
    # Move beginning from destination city
    while current_node != starting_city:
        route.insert(0, current_node)
        current_node = former_nodes[current_node]
    
    # Insert starting city to beginning
    route.insert(0, starting_city)
    
    # Print output
    print(f"{'-'.join(route)}, {shortest_route[destination_city]}")

In [6]:
calculateRoute(["Istanbul", "Ankara"], "distances.csv" )

Istanbul-Izmit-Adapazari-Bolu-Ankara, 468
